In [1]:
import os
import json
import numpy as np
import tensorflow as tf

In [2]:
# List that holds all the grids to be used as inputs for training
x_train_grid = []
# List that holds all the positions to be used as inputs for training
x_train_position = []
# List that holds all the distances to be used as inputs for training
x_train_distance = []
# List that holds all the locals to be used as inputs for training
x_train_locals = []
# List that holds all the directions to be used as outputs for training
y_train = []

# List that holds all the grids to be used as inputs for testing
x_test_grid = []
# List that holds all the positions to be used as inputs for testing
x_test_position = []
# List that holds all the distances to be used as inputs for testing
x_test_distance = []
# List that holds all the locals to be used as inputs for testing
x_test_locals = []
# List that holds all the directions to be used as outputs for testing
y_test = []

In [3]:
# Name of directory for grids
directory_name = './data/small_grids/'

direct_count = [0 for i in range(4)]

# iterate through all training grids
for file_name in os.listdir(directory_name):
    f = open(directory_name + file_name)
    data = json.load(f)

    # Iterate through all the data in a given grid and append their input and output values
    for i in data:
        x_train_grid.append(i['gridworld'])
        x_train_position.append(i['position'])
        x_train_locals.append(i['local'])
        y_train.append(i['direction'])
        direct_count[i['direction']] += 1
    
    # Close file socket
    f.close()

In [4]:
# Name of directory for grids
directory_name = './data/small_grids_test/'

# iterate through all training grids
for file_name in os.listdir(directory_name):
    f = open(directory_name + file_name)
    data = json.load(f)

    # Iterate through all the data in a given grid and append their input and output values
    for i in data:
        x_test_grid.append(i['gridworld'])
        x_test_position.append(i['position'])
        x_test_locals.append(i['local'])
        y_test.append(i['direction'])
    
    # Close file socket
    f.close()

In [5]:
# Reshape the data
train_in_grid = np.reshape( x_train_grid, (-1, 10, 10, 1) ) / 2
train_in_position = np.reshape( x_train_position, (-1, 2, 1, 1) )
train_in_locals = np.reshape( x_train_locals, (-1, 5, 5, 1) )
train_out = tf.keras.utils.to_categorical( y_train, 4 )

test_in_grid = np.reshape( x_test_grid, (-1, 10, 10, 1) ) / 2
test_in_position = np.reshape( x_test_position, (-1, 2, 1, 1) )
test_in_locals = np.reshape( x_test_locals, (-1, 5, 5, 1) )
test_out = tf.keras.utils.to_categorical( y_test, 4 )

In [6]:
# Flatten the grid input
grid_input = tf.keras.layers.Input( shape = (10,10,1) )
cnn_grid = tf.keras.layers.Conv2D( filters = 64, kernel_size = (5,5), strides = (1,1),
                               padding = "valid", activation = tf.nn.relu )( grid_input )
flatten_grid = tf.keras.layers.Flatten()( cnn_grid )

2021-12-12 03:37:12.779145: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
# Flatten the position input
position_input = tf.keras.layers.Input( shape = (2,1,1) )
cnn_position = tf.keras.layers.Conv2D( filters = 5, kernel_size = (1,1), strides = (1,1),
                               padding = "valid", activation = tf.nn.relu )( position_input )
flatten_position = tf.keras.layers.Flatten()( cnn_position )

In [8]:
# flatten the local inputs
local_input = tf.keras.layers.Input( shape = (5,5,1) )
cnn_local = tf.keras.layers.Conv2D( filters = 32, kernel_size = (3,3), strides = (1,1),
                               padding = "valid", activation = tf.nn.relu )( local_input )
flatten_local = tf.keras.layers.Flatten()( cnn_local )

In [9]:
# Concatenate the grid and position into one vector which will be passed to neural network as input
final_input = tf.keras.layers.Concatenate()([flatten_grid, flatten_local, flatten_position])
print(final_input)

KerasTensor(type_spec=TensorSpec(shape=(None, 2602), dtype=tf.float32, name=None), name='concatenate/concat:0', description="created by layer 'concatenate'")


In [10]:
# Create layers for Neural Network
dense_1 = tf.keras.layers.Dense( units = 50, activation = tf.nn.relu )( final_input )
dense_2 = tf.keras.layers.Dense( units = 10, activation = tf.nn.relu )( dense_1 )
logits = tf.keras.layers.Dense( units = 4, activation = None )( dense_2 )
probabilities = tf.keras.layers.Softmax()( logits )

In [11]:
# Compile the neural network to use stochastic gradient descent as the optimizer and categorical_crossentropy as loss function
model = tf.keras.Model( inputs = [grid_input, local_input, position_input], outputs = probabilities )
opt = tf.keras.optimizers.SGD()
model.compile( optimizer = opt, loss = 'categorical_crossentropy', metrics = ['accuracy'] )

In [12]:
def generate_confusion_matrix( data, labels ):
    mat = [ [ 0 for i in range(4) ] for j in range(4) ]
    
    prob_predict = model.predict( data )
    #print(prob_predict)
    predictions = np.argmax( prob_predict, axis = 1 )
    
    for i in range( data[0].shape[0] ):
        mat[ labels[i] ][ predictions[i] ] += 1
        # if labels[i] == 0:
        #     print(prob_predict[i])
    
    for i in range(4):
        print( "\t".join( [ str(c) for c in mat[i] ] ) )

In [13]:
# Test out before training
generate_confusion_matrix( [test_in_grid, test_in_locals, test_in_position], y_test )

0	0	37	0
0	0	160	0
0	0	643	0
0	0	947	0


In [14]:
# Train the model
history = model.fit( [train_in_grid, train_in_locals, train_in_position], train_out, validation_data=([test_in_grid, test_in_locals, test_in_position], test_out), epochs = 8 )

Epoch 1/8
472/472 [==============================] - 6s 9ms/step - loss: 0.4385 - accuracy: 0.8596 - val_loss: 0.1612 - val_accuracy: 0.9636
Epoch 2/8
472/472 [==============================] - 3s 7ms/step - loss: 0.1626 - accuracy: 0.9520 - val_loss: 0.1084 - val_accuracy: 0.9675
Epoch 3/8
472/472 [==============================] - 3s 6ms/step - loss: 0.1239 - accuracy: 0.9583 - val_loss: 0.0854 - val_accuracy: 0.9720
Epoch 4/8
472/472 [==============================] - 3s 6ms/step - loss: 0.1027 - accuracy: 0.9651 - val_loss: 0.0749 - val_accuracy: 0.9810
Epoch 5/8
472/472 [==============================] - 3s 7ms/step - loss: 0.0863 - accuracy: 0.9742 - val_loss: 0.0588 - val_accuracy: 0.9871
Epoch 6/8
472/472 [==============================] - 3s 6ms/step - loss: 0.0733 - accuracy: 0.9783 - val_loss: 0.0551 - val_accuracy: 0.9899
Epoch 7/8
472/472 [==============================] - 3s 7ms/step - loss: 0.0649 - accuracy: 0.9806 - val_loss: 0.0464 - val_accuracy: 0.9871
Epoch 8/8
472

In [15]:
# Test out after training
generate_confusion_matrix( [test_in_grid, test_in_locals, test_in_position], y_test )

30	3	2	2
1	155	3	1
0	0	642	1
1	1	1	944


In [16]:
model.save('./models/agent1_CNN_small')

2021-12-12 03:37:43.124479: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: ./models/agent1_CNN_small/assets
